# Notes to Improve Workflow

In [3]:
# Setup matplotlib to plot inline (within the notebook)
%matplotlib inline
# Import the pyplot module of Matplotlib as plt
import matplotlib.pyplot as plt
# Import pandas under the abbreviation 'pd'
import pandas as pd
# Import NumPy under the abbreviation 'np'
import numpy as np
# Libraries to aid in web scraping
import requests
from bs4 import BeautifulSoup

#Notes for efficienccy:
#shift + tab when cursor is in arguments to bring up its documentation
#ex: pd.DataFrame(Shift+tab here)
#recall: Json == Dictionary
#to know functions of pandas:
dir(pd)

['BooleanDtype',
 'Categorical',
 'CategoricalDtype',
 'CategoricalIndex',
 'DataFrame',
 'DateOffset',
 'DatetimeIndex',
 'DatetimeTZDtype',
 'ExcelFile',
 'ExcelWriter',
 'Flags',
 'Float32Dtype',
 'Float64Dtype',
 'Float64Index',
 'Grouper',
 'HDFStore',
 'Index',
 'IndexSlice',
 'Int16Dtype',
 'Int32Dtype',
 'Int64Dtype',
 'Int64Index',
 'Int8Dtype',
 'Interval',
 'IntervalDtype',
 'IntervalIndex',
 'MultiIndex',
 'NA',
 'NaT',
 'NamedAgg',
 'Period',
 'PeriodDtype',
 'PeriodIndex',
 'RangeIndex',
 'Series',
 'SparseDtype',
 'StringDtype',
 'Timedelta',
 'TimedeltaIndex',
 'Timestamp',
 'UInt16Dtype',
 'UInt32Dtype',
 'UInt64Dtype',
 'UInt64Index',
 'UInt8Dtype',
 '__all__',
 '__builtins__',
 '__cached__',
 '__deprecated_num_index_names',
 '__dir__',
 '__doc__',
 '__docformat__',
 '__file__',
 '__getattr__',
 '__git_version__',
 '__loader__',
 '__name__',
 '__package__',
 '__path__',
 '__spec__',
 '__version__',
 '_config',
 '_is_numpy_dev',
 '_libs',
 '_testing',
 '_typing',
 '_ve

# Reading The Data

In [4]:
# using pandas and read_csv()
acquis = pd.read_csv("datasets/startups/acquisitions.csv")
additions = pd.read_csv("datasets/startups/additions.csv")
companies = pd.read_csv("datasets/startups/companies.csv")
invests = pd.read_csv("datasets/startups/investments.csv")
rounds = pd.read_csv("datasets/startups/rounds.csv")
dfAll = [acquis, additions, companies, invests, rounds]

In [5]:
#shapes (rows x cols) of each dataset
for ele in dfAll:
    print(ele.shape)

(18969, 18)
(2213, 4)
(66368, 14)
(168647, 18)
(114949, 12)


# Samples of the datasets

In [4]:
acquis.sample(5).transpose()

,1678,6510,5569,13431,1917
company_permalink,/organization/authoria,/organization/franchisehelp,/organization/energov,/organization/provisionx,/organization/barcodesinc
company_name,Peopleclick Authoria,FranchiseHelp,EnerGov,ProvisionX,Barcodesinc
company_category_list,Software,Lead Generation|Performance Marketing|Small an...,NaN,NaN,Consumers|E-Commerce|Information Technology
company_country_code,USA,USA,USA,NaN,USA
company_state_code,MA,NY,GA,NaN,IL
company_region,Boston,New York City,Atlanta,NaN,Chicago
company_city,Waltham,New York,Duluth,NaN,Chicago
acquirer_permalink,/organization/bedford-funding,/organization/metric-collective,/organization/tyler-technologies,/organization/wmode,/organization/the-cortec-group
acquirer_name,Bedford Funding,Metric Collective,Tyler Technologies,Wmode,The Cortec Group
acquirer_category_list,Information Technology|SaaS|Services,NaN,Software,Telecommunications,NaN


In [5]:
additions.sample(5).transpose()

,109,1955,1579,1699,1404
content,Acquisitions,Rounds: seed,Rounds: grant,Rounds: post_ipo_equity,Rounds: convertible_note
month_str,2000-04,2011-02,2010-08,2009-01,2015-03
year_str,2000,2011,2010,2009,2015
value,11,90,10,1,130


In [6]:
companies.sample(5).transpose()

,4805,62816,11640,176,35560
permalink,/organization/atlas-watersystems,/organization/vtm,/organization/clew,/organization/2catalyze,/organization/metavine
name,Atlas Watersystems,VTM,Clew,2Catalyze,Metavine
homepage_url,http://www.atlaswatersystems.com/,http://vtm2020.com,http://www.clewllc.com,NaN,http://www.metavine.com
category_list,NaN,Biotechnology,Consulting|Market Research,Software,Application Platforms|Software
funding_total_usd,-,2015130,-,6785876,5500000
status,closed,operating,operating,operating,operating
country_code,USA,USA,USA,CAN,USA
state_code,MA,TN,MA,ON,CA
region,Boston,Nashville,Boston,Toronto,SF Bay Area
city,Waltham,Nashville,Lexington,Toronto,San Mateo


In [7]:
invests.sample(5).transpose()

,147020,101570,62744,118279,145661
company_permalink,/organization/theresumator,/organization/odesk,/organization/hacker-school,/organization/qunar-com,/organization/test-tv
company_name,Jazz,Upwork (formerly Elance-oDesk),Hacker School,Qunar.com,Test.tv
company_category_list,Human Resources|Internet|Recruiting|SaaS|Softw...,Crowdsourcing|Curated Web|Employment|Freelance...,Education,Business Services|Travel,Reviews and Recommendations|Shopping|Television
company_country_code,NaN,NaN,USA,CHN,RUS
company_state_code,NaN,NaN,NY,22,66
company_region,NaN,NaN,New York City,Beijing,St. Petersburg
company_city,NaN,NaN,New York,Beijing,Saint Petersburg
investor_permalink,/organization/ff-venture-capital,/organization/globespan-capital-partners,/organization/sv-angel,/organization/mayfield-fund,/organization/rsv-venture-partners
investor_name,ff Venture Capital,Globespan Capital Partners,SV Angel,Mayfield Fund,RSV Venture Partners
investor_country_code,USA,USA,USA,USA,NaN


In [8]:
rounds.sample(5).transpose()

,114692,73177,33448,20598,55925
company_permalink,/organization/zounds,/organization/paratek,/organization/everwise,/organization/cleverset,/organization/lijit-networks
company_name,Zounds,Paratek,Everwise,CleverSet,Lijit Networks
company_category_list,Batteries|Email|Forums|Franchises|Hardware + S...,Mobile,Human Resources|SaaS|Software,E-Commerce|Reviews and Recommendations|Shopping,Advertising
company_country_code,USA,USA,USA,USA,USA
company_state_code,AZ,NH,NY,WA,CO
company_region,Phoenix,"Manchester, New Hampshire",New York City,Seattle,Denver
company_city,Phoenix,Nashua,New York,Seattle,Boulder
funding_round_permalink,/funding-round/e7b02816ca33e195f0fcff2504ea16bb,/funding-round/caf1ccde060a61d122771737b4b43103,/funding-round/7c0d103bbf09b677afd294076e70bc58,/funding-round/6d4839b6f05ca1555ec39cf6da4fec63,/funding-round/68f8d13b2b7007e19c64504461983490
funding_round_type,venture,venture,seed,venture,venture
funding_round_code,NaN,B,NaN,A,E


# Seeing the columns in the datasets

In [9]:
def datasetsCols(dfList, datasetNames, pad=""):
    """
    Takes dataframes and returns a dataframe (df) with each column having a df's
    column names. The "pad" is to make sure all lists of columns have 
    the same length padded out with "pad"
    Example
    pad = "XX", datasetNames = ["cars", "planes"]
    dfList = [carsDF, planesDF]
    returned data frame:
            cars            planes
    0       numOfWheels     numOfWings
    1       manufacturer    manufacturer
    2       make            XX
    """
    cols = []
    maxArrayLen = 0
    for i, df in enumerate(dfList):
        cols.append(df.columns.tolist())
        maxArrayLen = max(maxArrayLen, len(cols[i]))
    
    dictCsvs = {}
    for i, df in enumerate(dfList):
        cols[i] += [pad] * (maxArrayLen - len(cols[i])) #padding the lists to make them have equal lengths 
        dictCsvs.update({datasetNames[i] : cols[i]})

    return pd.DataFrame(dictCsvs)

In [10]:
lstCsvNames = ["acquisitions", "additions", "companies", "investments", "rounds"]

dfAllCols = datasetsCols(dfAll, lstCsvNames)
dfAllCols

,acquisitions,additions,companies,investments,rounds
0,company_permalink,content,permalink,company_permalink,company_permalink
1,company_name,month_str,name,company_name,company_name
2,company_category_list,year_str,homepage_url,company_category_list,company_category_list
3,company_country_code,value,category_list,company_country_code,company_country_code
4,company_state_code,,funding_total_usd,company_state_code,company_state_code
5,company_region,,status,company_region,company_region
6,company_city,,country_code,company_city,company_city
7,acquirer_permalink,,state_code,investor_permalink,funding_round_permalink
8,acquirer_name,,region,investor_name,funding_round_type
9,acquirer_category_list,,city,investor_country_code,funding_round_code


# Data Cleaning

First, lets change the columns in the `companies` dataset <br>
to be the similar to all other datasets: <br>
(we're doing this in case we merge the datasets together, we want them to have the same column names)

In [13]:
companies.rename(columns = {'permalink' : 'company_permalink', 
                            'name' : 'company_name',
                            'category_list' : 'company_category_list',
                            'country_code' : 'company_country_code',
                            'state_code' : 'company_state_code',
                            'region' : 'company_region',
                            'city' : 'company_city'}, inplace = True)

In [14]:
dfAll[2] = companies
dfAllCols = datasetsCols(dfAll, lstCsvNames)
dfAllCols

NameError: name 'datasetsCols' is not defined

Let's start by analyzing `additions` dataset, as it has unusual format of columns

## Removing "additions" Dataset

In [13]:
additions.sample(5)

,content,month_str,year_str,value
882,Profiles: Organizations,2010-11,2010,2163
1857,Rounds: product_crowdfunding,2013-09,2013,26
133,Acquisitions,2002-04,2002,8
75,Acquisitions,1997-06,1997,7
1102,Profiles: Products,2011-11,2011,215


In [14]:
additions["content"].unique()

array(['Acquisitions', 'IPOs', 'Profiles', 'Profiles: Organizations',
       'Profiles: People', 'Profiles: Products', 'Rounds',
       'Rounds: angel', 'Rounds: convertible_note',
       'Rounds: debt_financing', 'Rounds: equity_crowdfunding',
       'Rounds: grant', 'Rounds: non_equity_assistance',
       'Rounds: post_ipo_debt', 'Rounds: post_ipo_equity',
       'Rounds: private_equity', 'Rounds: product_crowdfunding',
       'Rounds: secondary_market', 'Rounds: seed', 'Rounds: undisclosed',
       'Rounds: venture'], dtype=object)

In [15]:
dfAllCols

,acquisitions,additions,companies,investments,rounds
0,company_permalink,content,company_permalink,company_permalink,company_permalink
1,company_name,month_str,company_name,company_name,company_name
2,company_category_list,year_str,homepage_url,company_category_list,company_category_list
3,company_country_code,value,company_category_list,company_country_code,company_country_code
4,company_state_code,,funding_total_usd,company_state_code,company_state_code
5,company_region,,status,company_region,company_region
6,company_city,,company_country_code,company_city,company_city
7,acquirer_permalink,,company_state_code,investor_permalink,funding_round_permalink
8,acquirer_name,,company_region,investor_name,funding_round_type
9,acquirer_category_list,,company_city,investor_country_code,funding_round_code


In [16]:
additions.shape

(2213, 4)

In [17]:
companies.shape, rounds.shape

((66368, 14), (114949, 12))

So apparently there are no columns that could be used to join with the other datasets. <br>
Furthermore, the "value" column is too vague to be useful. <br>
Therefore, the "additions" dataset will be discarded.

In [18]:
dfAll.pop(1)
lstCsvNames.pop(1)
len(dfAll), lstCsvNames

(4, ['acquisitions', 'companies', 'investments', 'rounds'])

# Data Cleaning (Cont.)

## Checking All Missing Values

In [19]:
def datasetsNulls(dfList, datasetNames, nullCol="nulls_", pad=""):
    """
    Use this when you want to display a column of column names,
    then a column of the null values, and repeat that for each dataset.
    Returns a dataframe
    """
    cols = []
    nulls = []
    maxArrayLen = 0
    for i, df in enumerate(dfList):
        cols.append(df.columns.tolist())
        nulls.append(df.isnull().sum().tolist())
        maxArrayLen = max(maxArrayLen, len(cols[i]))
    
    dictCsvs = {}
    for i, df in enumerate(dfList):
        cols[i] += [pad] * (maxArrayLen - len(cols[i])) #padding the lists to make them have equal lengths 
        nulls[i] += [None] *  (maxArrayLen - len(nulls[i]))
        dictCsvs[datasetNames[i]] = cols[i]
        dictCsvs[nullCol + str(i+1)] = nulls[i]
    
    return pd.DataFrame(dictCsvs)

In [20]:
dfAllNulls = datasetsNulls(dfAll, lstCsvNames)
dfAllNulls

,acquisitions,nulls_1,companies,nulls_2,investments,nulls_3,rounds,nulls_4
0,company_permalink,0,company_permalink,0.0,company_permalink,0,company_permalink,0.0
1,company_name,1,company_name,1.0,company_name,1,company_name,1.0
2,company_category_list,4984,homepage_url,5058.0,company_category_list,3632,company_category_list,3410.0
3,company_country_code,4669,company_category_list,3148.0,company_country_code,10521,company_country_code,8678.0
4,company_state_code,4949,funding_total_usd,0.0,company_state_code,13431,company_state_code,10946.0
5,company_region,4911,status,0.0,company_region,12485,company_region,10167.0
6,company_city,4911,company_country_code,6958.0,company_city,12483,company_city,10164.0
7,acquirer_permalink,1,company_state_code,8547.0,investor_permalink,12,funding_round_permalink,0.0
8,acquirer_name,1,company_region,8030.0,investor_name,12,funding_round_type,0.0
9,acquirer_category_list,2389,company_city,8028.0,investor_country_code,30939,funding_round_code,83809.0


### `dropna()` for columns with few missing values

Notice that there is one row that doesn't contain a company name, <br>
so let's validate that it is the same company across all datasets, <br>
so we can remove it:

In [21]:
datasetsCols(dfAll, lstCsvNames)

,acquisitions,companies,investments,rounds
0,company_permalink,company_permalink,company_permalink,company_permalink
1,company_name,company_name,company_name,company_name
2,company_category_list,homepage_url,company_category_list,company_category_list
3,company_country_code,company_category_list,company_country_code,company_country_code
4,company_state_code,funding_total_usd,company_state_code,company_state_code
5,company_region,status,company_region,company_region
6,company_city,company_country_code,company_city,company_city
7,acquirer_permalink,company_state_code,investor_permalink,funding_round_permalink
8,acquirer_name,company_region,investor_name,funding_round_type
9,acquirer_category_list,company_city,investor_country_code,funding_round_code


In [22]:
#company_permalink of each row that doesn't have a company name:
for df in dfAll:
    print(df[df['company_name'].isnull()]['company_permalink'].values)

['mendatio"']
['/organization/tell-it-in']
['/organization/tell-it-in']
['/organization/tell-it-in']


So apparently, the `acquisitions` dataset has a different permalink that has no company name, <br>
So we could remove these two companies, as the total number of companies are big enough to get the insight that we want.

`dfAllNulls` dataframe also shows that there are missing values for `acquirer_name` and `investor_name` <br>
which can't be imputed, so we'll also remove them.

Finally, we'll remove the missing values for columns that have less than 100 `NaN`, <br>
as they're small in comparison to their respective datasets:

In [23]:
for df in dfAll:
    nulls = df.isnull().sum()
    cols = nulls[(nulls <= 100) & (nulls != 0)].index.tolist()
    df.dropna(subset=cols, inplace=True)
dfAllNulls = datasetsNulls(dfAll, lstCsvNames)
dfAllNulls

,acquisitions,nulls_1,companies,nulls_2,investments,nulls_3,rounds,nulls_4
0,company_permalink,0,company_permalink,0.0,company_permalink,0,company_permalink,0.0
1,company_name,0,company_name,0.0,company_name,0,company_name,0.0
2,company_category_list,4977,homepage_url,5058.0,company_category_list,3632,company_category_list,3410.0
3,company_country_code,4661,company_category_list,3147.0,company_country_code,10521,company_country_code,8678.0
4,company_state_code,4940,funding_total_usd,0.0,company_state_code,13431,company_state_code,10946.0
5,company_region,4902,status,0.0,company_region,12485,company_region,10167.0
6,company_city,4902,company_country_code,6956.0,company_city,12483,company_city,10164.0
7,acquirer_permalink,0,company_state_code,8545.0,investor_permalink,0,funding_round_permalink,0.0
8,acquirer_name,0,company_region,8028.0,investor_name,0,funding_round_type,0.0
9,acquirer_category_list,2387,company_city,8026.0,investor_country_code,30927,funding_round_code,83808.0


## Converting & Imputing Dates

[SAY WHAT YOU WILL DO HERE FARAH:]

In [24]:
#Farah: plz impute (not drop) the columns' missing values related to dates in each dataset which are:
#founded_at.. and that's it :] 
# Suggestion: convert the dates of all the columns that have date values from "object" to datetime
#       Helpful link: https://pandas.pydata.org/pandas-docs/stable/user_guide/timeseries.html#converting-to-timestamps
# Then, make the founded_at a couple of days before first_funding_at

#Sincerely No. -Farah :)

We are currently interested in 'founded_at', 'first_funding_at' and 'last_funding_at' columns. Before doing operations on dates, they need to be converted first from 'Object' to 'DateTime'.

In [25]:
companies['founded_at'] =pd.to_datetime(companies['founded_at'], errors = 'coerce')
companies['first_funding_at'] =pd.to_datetime(companies['first_funding_at'], errors = 'coerce')
companies['last_funding_at'] =pd.to_datetime(companies['last_funding_at'], errors = 'coerce')
companies.dtypes

company_permalink                object
company_name                     object
homepage_url                     object
company_category_list            object
funding_total_usd                object
status                           object
company_country_code             object
company_state_code               object
company_region                   object
company_city                     object
funding_rounds                    int64
founded_at               datetime64[ns]
first_funding_at         datetime64[ns]
last_funding_at          datetime64[ns]
dtype: object

After converting, we need to impute the missing values on the dates of when those companies where founded at. So, this will be done by first creating a new column that calculates how long it took to get the first funding. This will be done by subtracting the date of the founding from the date of the first funding. Note that we will find some values in negative which indicate that some companies took funding before establishing it.


In [26]:
companies['time_before_first_fund'] = companies['first_funding_at'] - companies['founded_at']
companies['time_before_first_fund']

0             NaT
1       -187 days
2             NaT
3        443 days
4       1665 days
           ...   
66363         NaT
66364   -194 days
66365   1308 days
66366    365 days
66367         NaT
Name: time_before_first_fund, Length: 66343, dtype: timedelta64[ns]

Now we need to calculate the mean of the time before the first funding and impute the missing values with this average. Notice that the average is 1370 days which is around 3.7 years which means there must be some outliers that messed up the average.

In [27]:
companies['time_before_first_fund'].mean()

0       1370 days 03:49:18.015610624
1                -187 days +00:00:00
2       1370 days 03:49:18.015610624
3                  443 days 00:00:00
4                 1665 days 00:00:00
                    ...             
66363   1370 days 03:49:18.015610624
66364            -194 days +00:00:00
66365             1308 days 00:00:00
66366              365 days 00:00:00
66367   1370 days 03:49:18.015610624
Name: time_before_first_fund, Length: 66343, dtype: timedelta64[ns]

Therefore, we will need to visualize those outliers and maybe if they are causing inaccuracy in the data, we can drop them.

In [28]:
plt.scatter(companies['first_funding_at'], companies['founded_at'], s = 20, alpha = 0.5)
plt.xlabel('first_funding_at')
plt.ylabel('founded_at')

0       2011-04-05 20:10:41.984389376
1       2014-09-04 00:00:00.000000000
2       2010-04-30 20:10:41.984389376
3       2007-01-01 00:00:00.000000000
4       2010-01-01 00:00:00.000000000
                     ...             
66363   2008-06-30 20:10:41.984389376
66364   2012-05-13 00:00:00.000000000
66365   2011-01-01 00:00:00.000000000
66366   2014-01-01 00:00:00.000000000
66367   2009-12-30 20:10:41.984389376
Name: founded_at, Length: 66343, dtype: datetime64[ns]

We Noticed that there are some outliers than can be removed from the dataset so we will create a temporary dataframe that will contain the outliers so we can drop them later on. The first outliers that we want to deduct are the ones at the left of the center.

In [ ]:
tempdf = companies.copy()
tempdf = tempdf[(tempdf.first_funding_at <= datetime.strptime("01/01/1983", "%d/%m/%Y"))& (tempdf.founded_at <= datetime.strptime("01/01/2000", "%d/%m/%Y"))]
dropseries = tempdf.copy()
dropseries

Plotting the outliers on a separate graph:

In [ ]:
plt.scatter(tempdf['first_funding_at'], tempdf['founded_at'], s = 20)
plt.xlabel('first_funding_at')
plt.ylabel('founded_at')

Now we will determine the outliers that are the bottom of the center from the original graph.

In [ ]:
tempdf = companies.copy()
tempdf = tempdf[(tempdf.first_funding_at < datetime.strptime("01/01/2005", "%d/%m/%Y"))& (tempdf.founded_at <= datetime.strptime("01/01/1900", "%d/%m/%Y"))]
tempdf

Plotting the outliers on a separate graph:

In [ ]:
plt.scatter(tempdf['first_funding_at'], tempdf['founded_at'], s = 20)
plt.xlabel('first_funding_at')
plt.ylabel('founded_at')

Now we will concatenate the newly discovered outliers to the ones from before

In [ ]:
dropseries = pd.concat([dropseries, tempdf])
dropseries

Finally, we will deduct the outliers that are the top right corner

In [ ]:
tempdf = companies.copy()
tempdf = tempdf[(tempdf.first_funding_at > datetime.strptime("01/01/2010", "%d/%m/%Y"))& (tempdf.founded_at >= datetime.strptime("01/01/2030", "%d/%m/%Y"))]
tempdf

Visualizing the outliers:

In [ ]:
plt.scatter(tempdf['first_funding_at'], tempdf['founded_at'], s = 20)
plt.xlabel('first_funding_at')
plt.ylabel('founded_at')

Concatinating the outliers on the rest:

In [ ]:
dropseries = pd.concat([dropseries, tempdf])
dropseries

Now, we can drop all of the outliers that have been detected.


In [ ]:
list = dropseries.index
companies = companies.drop(list)
#companies['time_before_first_fund'].sort_values(ascending=False)

Plotting the graph once more after cleaning:

In [ ]:
plt.scatter(companies['first_funding_at'], companies['founded_at'], s = 20, alpha=0.5)
plt.xlabel('first_funding_at')
plt.ylabel('founded_at')

Now, we can calculate the mean. Notice that there were not much of a difference but deducting the outliers helped keeping the data realistic because some dates were 2090, 2100, etc. which is unrealistic.

In [ ]:
companies['time_before_first_fund'].mean()

Filling the missing values with the new mean:

In [ ]:
companies['time_before_first_fund'].fillna(companies['time_before_first_fund'].mean(), inplace = True)
companies['time_before_first_fund']

After calculating the mean and filling the nulls with it, now we can impute the missing dates in the 'founded_at' column by subtracting the time before first funding from the first funding date.

In [ ]:
companies['founded_at'].fillna(companies['first_funding_at'] - companies['time_before_first_fund'], inplace = True)
companies['founded_at']

Converting 'aquired_at' of the acquisitons dataset from Object to date:

In [ ]:
acquis['acquired_at'] =pd.to_datetime(acquis['acquired_at'], errors = 'coerce')
acquis.dtypes

Converting 'funded_at' of the investments dataset from Object to date:

In [ ]:
invests['funded_at'] =pd.to_datetime(invests['funded_at'], errors = 'coerce')
invests.dtypes

Converting 'funded_at' of the rounds dataset from Object to date:

In [ ]:
rounds['funded_at'] =pd.to_datetime(rounds['funded_at'], errors = 'coerce')
rounds.dtypes

## Imputing Categories

Regarding the category list in the five datasets, since we don't have any indicator on deducing the null values in this column, we will fill the nulls with "missing". With samples, it's provided that the same rows (companies) that their category is missing in one dataset, are the same rows in the others so even merging the datasets will not be helpful.


In [ ]:
companies['company_category_list'].fillna('missing', inplace=True)
acquis['company_category_list'].fillna('missing', inplace=True)
acquis['acquirer_category_list'].fillna('missing', inplace=True)
invests['company_category_list'].fillna('missing', inplace=True)
rounds['company_category_list'].fillna('missing', inplace=True)

In [29]:
#farah stop point here

## Cleaning & Imputing Geographical Locations

### Removing `company_state_code` and `company_region`

We don't need `company_state_code` in our analysis, <br>
As country code and city are sufficient to know <br>
an approximation of the geographical location of the company. <br>
Therefore, let's remove it:

In [30]:
companies[['company_country_code', 'company_state_code', 'company_city']].head(10)

,company_country_code,company_state_code,company_city
0,IND,16,Mumbai
1,USA,DE,Delaware City
2,NaN,NaN,NaN
3,CHN,22,Beijing
4,USA,IL,Champaign
5,HKG,NaN,Hong Kong
6,CAN,BC,Vancouver
7,USA,CA,Mountain View
8,USA,CA,San Francisco
9,CHL,12,Las Condes


In [31]:
companies.drop('company_state_code', axis=1, inplace=True)

`company_region` is also redundent, as it is usually the same as `company_city`.<br>
Let's validate this:

In [32]:
# Note: same logic applies for the permalinks
acquis.dropna(subset=['acquirer_permalink', 'acquirer_name'], inplace=True)
invests.dropna(subset=['investor_permalink', 'investor_name'], inplace=True)

Also, we'll remove the missing values for columns that have less than 100 `NaN`, <br>
as they're small in comparison to their respective datasets:

In [33]:
companies[companies.company_region == companies.company_city][['company_region', 'company_city']]

,company_region,company_city
0,Mumbai,Mumbai
3,Beijing,Beijing
5,Hong Kong,Hong Kong
6,Vancouver,Vancouver
11,Orlando,Orlando
...,...,...
66354,Bath,Bath
66360,Milwaukee,Milwaukee
66362,London,London
66363,Beijing,Beijing


Let's see the mean of similarity between the strings using builtin library `difflib.SequenceMatcher`:

In [34]:
from difflib import SequenceMatcher
similarities = []
rgs = companies.company_region.values.tolist()
cts = companies.company_city.values.tolist()
for i in range(min(len(rgs), len(cts))):
    if not (rgs[i] != rgs[i] or cts[i] != cts[i]): # If either of the elements is NaN, then don't find the similarity
        similarities.append(SequenceMatcher(None, rgs[i], cts[i]).ratio()) # Gets the similarity between each two strings
avgSim = sum(similarities) / len(similarities)
avgSim

0.6175093624179544

However, `SequenceMatcher` only finds the longest contiguous matching subsequence, <br>
and all other characters will drastically decrease the similarity, which isn't very accurate, <br>
Therefore, let's try to see how many times a string is a substring of another:

In [35]:
subsCount = 0
for i in range(min(len(rgs), len(cts))):
    if not (rgs[i] != rgs[i] or cts[i] != cts[i]):
        subsCount += (rgs[i] in cts[i] or cts[i] in rgs[i])
subsCount

29884

So to summarize: almost half of the companies have the same data for `region` and `city` (25360) <br>
There are around 5000 companies that have the same semantic meaning between the data <br>
(e.g: 'New York City' is the same as 'NYC, New York City') <br>
Therefore, `company_region` can be removed without losing any possible future insights:

In [36]:
companies.drop('company_region', axis=1, inplace=True)

### Initial Failed Method: Imputing `company_country_code` and `company_city` via `company_permalink`

Steps to impute country codes is to do the following for all missing values:
1. check the company's permalink
2. find HTML that contains the headquarters location (country and city)
3. scrape country and convert it to country code using `pycountry.countries`
4. scrape city and impute it in `company_city`

Step 1: appending "https://www.crunchbase.com" to each permalink <br>
to be able to access the company's webpage on crunchbase. <br>
However, since the new permalink will be much longer (thus slower for accessing as a dataset's primary index) <br>
we'll create a function that appends the string as a prefix and that will be accessed throughout the notebook

In [37]:
##################################### function "visitLink()" used throughout the notebook #####################################
visitLink = lambda s : "https://www.crunchbase.com" + s

Step 2: Finding location of country and city on the webpage:
let's check company [004](https://www.crunchbase.com/organization/004)'s webpage for example: <Br>
<img src="Phase 1/countryAndCityInCrunchbase.png" width=400 height=300 />

By inspecting the HTML, we notice 4 instances of the class <br>
`component--field-formatter field-type-identifier-multi` (in a `span` element),<br>and the data that we want
is always the first instance: <br>
<img src="Phase 1/countryAndCityLocationInHTML.png" width=700 height=500 />

Step 3: scrape that info using `requests` and `bs4` libraries: <br>
(Note, we're using `threading.Thread` here to increase performance by scraping in multithreads)

However, this is a deadend, as crunchbase is powered by [distill network](https://www.google.com/search?q=distil+network&sxsrf=APq-WBt2mubTHz1xGfmKgpzlWtHah5qAaA%3A1648578960023&ei=kFFDYsx646qVB_nKh7gK&ved=0ahUKEwjMgq-a--v2AhVjVeUKHXnlAacQ4dUDCA4&uact=5&oq=distil+network&gs_lcp=Cgdnd3Mtd2l6EAMyBggAEAcQHjIGCAAQBxAeMgYIABAHEB4yBggAEAcQHjIGCAAQBxAeMgYIABAHEB4yBggAEAcQHjIGCAAQBxAeMgYIABAHEB4yBggAEAcQHjoHCCMQsAMQJzoHCAAQRxCwA0oECEEYAEoECEYYAFCyCliyCmCLD2gBcAF4AIABowGIAaMBkgEDMC4xmAEAoAEByAEKwAEB&sclient=gws-wiz#:~:text=distil%20networks%20uses%20machine%20learning%20to%20identify%20and%20mitigate%20potential%20bad%20bots%2C%20fingerprinting%20them%20so%20that%20they%20can%20still%20be%20tracked%20if%20they%20reconnect%20from%20a%20different%20ip%20address) <br>
So when we tried to scrap from it, the following html was always displayed: <br><br>
<img src="Phase 1/crunchbaseAccessDenied.png">

Step 3 failed-alternative: use linkedin to scrap the country codes and cities, <br>
but that also didn't work because linkedin displays javascript code note HTML so `requests` won't work here <br>
Even when we tried `selenium` it didn't work

In [8]:
headers = {
'authority': 'scrapeme.live',
'dnt': '1',
'upgrade-insecure-requests': '1',
'user-agent': 'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_15_4) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/83.0.4103.61 Safari/537.36',
'accept': 'text/html,application/xhtml+xml,application/xml;q=0.9,image/webp,image/apng,*/*;q=0.8,application/signed-exchange;v=b3;q=0.9',
'sec-fetch-site': 'none',
'sec-fetch-mode': 'navigate',
'sec-fetch-user': '?1',
'sec-fetch-dest': 'document',
'accept-language': 'en-GB,en-US;q=0.9,en;q=0.8',
}

In [183]:
from geopy.geocoders import Nominatim
import pycountry
geolocator = Nominatim(user_agent="ashraf196280@bue.edu.eg")
location = geolocator.geocode("Alexandria",language="en")
country = location.address.split(', ')[-1]
alpha3Code = pycountry.countries.get(name=country).alpha_3
country, alpha3Code

('Egypt', 'EGY')

In [ ]:
from linkedin_scraper import Company, actions
from selenium import webdriver

options=webdriver.ChromeOptions()
options.add_argument('--incognito')
options.add_argument('--headless')
driver=webdriver.Chrome(options=options)
driver.get('https://www.linkedin.com/uas/login')

username = driver.find_element_by_id('username')
username.send_keys('ashraf196280@bue.edu.eg')
password = driver.find_element_by_id('password')
password.send_keys('0LinkedIn0')
log_in_button = driver.find_element_by_class_name('from__button--floating')
log_in_button.click()

urls=['https://www.linkedin.com/company/10tracks/about/']
for url in urls:
    driver.get(url)
    time.sleep(3)
    soup = BeautifulSoup(driver.page_source, 'html.parser')


In [ ]:
dl = soup.find('div', 'mb6').find('dl')
nms = []
for tag in dl:
    txt = tag.text
    nms.append(txt)
nms

['\n',
 '\n          Website\n        ',
 '\n',
 '\n\n\n              http://10tracks.com\n            \n\n',
 '\n',
 '',
 '\n',
 '\n            Industry\n          ',
 '\n',
 '\n            Internet Publishing\n          ',
 '\n',
 '\n          Company size\n        ',
 '\n',
 '\n            2-10 employees\n        ',
 '\n',
 '\n            3 on LinkedIn\n            \n \n\n\n\n    Includes members with current employer listed as 10tracks, including part-time roles.\n\n\n\n',
 '\n',
 '\n            Headquarters\n          ',
 '\n',
 '\n            Moscow, Moscow\n          ',
 '\n',
 '',
 '\n',
 '',
 '\n',
 '',
 ' ']

In [ ]:
dl = soup.find('div', 'mb6').find('dl')
keys = []
values = []
for tag in dl.find_all('dt'):
    txt = tag.text.strip()
    if (txt != ''):
        keys.append(txt)
for tag in dl.find_all('dd'):
    txt = tag.text.strip()
    if (txt != ''):
        values.append(txt)


In [ ]:
keys, values

(['Website', 'Industry', 'Company size', 'Headquarters'],
 ['http://10tracks.com',
  'Internet Publishing',
  '2-10 employees',
  '3 on LinkedIn\n            \n \n\n\n\n    Includes members with current employer listed as 10tracks, including part-time roles.',
  'Moscow, Moscow'])

In [ ]:
values = [txt for txt in values if "Includes" not in txt]
values

['http://10tracks.com',
 'Internet Publishing',
 '2-10 employees',
 'Moscow, Moscow']

In [ ]:
linkedinDict = dict(zip(keys, values))
linkedinDict

{'Website': 'http://10tracks.com',
 'Industry': 'Internet Publishing',
 'Company size': '2-10 employees',
 'Headquarters': 'Moscow, Moscow'}

In [187]:
linkedinDict["Headquarters"]

'Moscow, Moscow'

In [188]:
import threading
lock = threading.Lock()
import pycountry
import time
from geopy.geocoders import Nominatim
import undetected_chromedriver as uc


def fetchCountry(idx, link):
    try:
        options=webdriver.ChromeOptions()
        options.add_argument('--incognito')
        options.add_argument('--headless')
        options.add_argument('log-level=1')
        driver=webdriver.Chrome(options=options)
        driver.get('https://www.linkedin.com/uas/login')

        username = driver.find_element_by_id('username')
        username.send_keys('ashraf196280@bue.edu.eg')
        password = driver.find_element_by_id('password')
        password.send_keys('0LinkedIn0')
        log_in_button = driver.find_element_by_class_name('from__button--floating')
        log_in_button.click()
        time.sleep(1)
        driver.get(link)
        time.sleep(2)
        soup = BeautifulSoup(driver.page_source, "html.parser")

        dl = soup.find('div', 'mb6').find('dl')
        keys = []
        values = []
        for tag in dl.find_all('dt'):
            txt = tag.text.strip()
            if (txt != ''):
                keys.append(txt)
        for tag in dl.find_all('dd'):
            txt = tag.text.strip()
            if (txt != ''):
                values.append(txt)
        values = [txt for txt in values if "Includes" not in txt]
        linkedinDict = dict(zip(keys, values))
        print(1)
        cityAndState = linkedinDict["Headquarters"]
        print(cityAndState)
    except:
        print("missing")
        lock.acquire()
        #companies.at[idx, 'company_country_code'] = "missing" # means that this website is not found on linkedin, so declare it as missing
        lock.release()
        return

    city = cityAndState.split(',')[0] # eg: "Cupertino, California" will be ["Cupertino", "California"], so "Cupertino" will be returned
    geolocator = Nominatim(user_agent="ashraf196280@bue.edu.eg")
    location = geolocator.geocode(city, language="en")
    country = location.address.split(', ')[-1]
    alpha3Code = pycountry.countries.get(name=country).alpha_3
    print(alpha3Code)
    lock.acquire()
    #companies.at[idx, 'company_country_code'] = alpha3Code # For later uses: "companies" will be replaced by your "global" dataframe
    lock.release()


In [189]:
from threading import Thread
def imputeFromLinks(df, colToSuffixIntoURL, colToImpute, linkPrefix = ""):
    linkss = df[df[colToImpute].isnull()][[colToSuffixIntoURL]].squeeze() # "squeeze()" converts df to series
    links = linkss[0:5]
    threads = []
    for tup in links.iteritems(): # tup[0] --> index, tup[1] --> url (i.e. link)
        th = Thread(target=fetchCountry, args=(tup[0], linkPrefix + tup[1].replace(" ", "") + '/about')) #making sure company name doesn't have spaces when suffixed into the url
        threads.append(th)
        th.start()
    for th in threads:
        th.join() # to wait until all multithreads finish to properly display the dataframe

In [190]:
imputeFromLinks(companies, "company_name", "company_country_code", "https://www.linkedin.com/company/")


C:\Users\Gamma\AppData\Local\Temp\ipykernel_39004\1105127303.py:18: DeprecationWarning: find_element_by_* commands are deprecated. Please use find_element() instead
  username = driver.find_element_by_id('username')
C:\Users\Gamma\AppData\Local\Temp\ipykernel_39004\1105127303.py:20: DeprecationWarning: find_element_by_* commands are deprecated. Please use find_element() instead
  password = driver.find_element_by_id('password')
C:\Users\Gamma\AppData\Local\Temp\ipykernel_39004\1105127303.py:22: DeprecationWarning: find_element_by_class_name is deprecated. Please use find_element(by=By.CLASS_NAME, value=name) instead
  log_in_button = driver.find_element_by_class_name('from__button--floating')


missing
missing
missing
missing
missing
missing
1
Moscow, Moscow


Exception in thread Thread-48:
Traceback (most recent call last):
  File "C:\Users\Gamma\AppData\Local\Programs\Python\Python38\lib\threading.py", line 932, in _bootstrap_inner
    self.run()
  File "C:\Users\Gamma\AppData\Local\Programs\Python\Python38\lib\threading.py", line 870, in run
    self._target(*self._args, **self._kwargs)
  File "C:\Users\Gamma\AppData\Local\Temp\ipykernel_39004\1105127303.py", line 56, in fetchCountry
AttributeError: 'NoneType' object has no attribute 'alpha_3'


1
missing


In [116]:
linkss = companies[companies['company_country_code'].isnull()][['company_name']].squeeze()
linkss[0:5]

2                  (THE) ONE of THEM,Inc.
21    Beijing 1000CHI Software Technology
35                           Beans Around
39                               10tracks
45                      115 network disks
Name: company_name, dtype: object

In [18]:
companies[companies.company_country_code.isnull() == True].shape

(6958, 14)

In [23]:
#checking current nulls
companies[companies.company_country_code.isnull() == True].sample(5)

,company_permalink,company_name,homepage_url,company_category_list,funding_total_usd,status,company_country_code,company_state_code,company_region,company_city,funding_rounds,founded_at,first_funding_at,last_funding_at
61754,/organization/viap,VIAP,http://www.viap.tv,Software,850000,closed,NaN,NaN,NaN,NaN,2,8/1/2010,4/1/2011,9/1/2011
17362,/organization/edition-f,EDITION F GmbH,http://www.editionf.com,Social Media,-,operating,NaN,NaN,NaN,NaN,1,2/1/2014,5/8/2014,5/8/2014
32780,/organization/livejasminbook,LiveJasminbook,http://www.livejasminbook.com,Chat|SexTech|Social Network Media,100000,operating,NaN,NaN,NaN,NaN,1,NaN,9/12/2014,9/12/2014
54841,/organization/storeage-networking-technologies,StoreAge Networking Technologies,NaN,NaN,25000000,closed,NaN,NaN,NaN,NaN,1,NaN,1/25/2001,1/25/2001
395,/organization/5-cups-and-some-sugar,5 CUPS and some sugar,http://5cups.de,Curated Web,385980,operating,NaN,NaN,NaN,NaN,1,1/1/2012,7/23/2013,7/23/2013


## Checking All `dtypes`

In [ ]:
def datasetsDtypes(dfList, datasetNames, dtypesCol="dtypes_", pad=""):
    cols = []
    nulls = []
    maxArrayLen = 0
    for i, df in enumerate(dfList):
        cols.append(df.columns.tolist())
        nulls.append(df.dtypes.tolist())
        maxArrayLen = max(maxArrayLen, len(cols[i]))
    
    dictCsvs = {}
    for i, df in enumerate(dfList):
        cols[i] += [pad] * (maxArrayLen - len(cols[i])) #padding the lists to make them have equal lengths 
        nulls[i] += [None] *  (maxArrayLen - len(nulls[i]))
        dictCsvs[datasetNames[i]] = cols[i]
        dictCsvs[dtypesCol + str(i+1)] = nulls[i]
    
    return pd.DataFrame(dictCsvs)

In [ ]:
datasetsDtypes(dfAll, lstCsvNames)

,acquisitions,dtypes_1,companies,dtypes_2,investments,dtypes_3,rounds,dtypes_4
0,company_permalink,object,company_permalink,object,company_permalink,object,company_permalink,object
1,company_name,object,company_name,object,company_name,object,company_name,object
2,company_category_list,object,homepage_url,object,company_category_list,object,company_category_list,object
3,company_country_code,object,company_category_list,object,company_country_code,object,company_country_code,object
4,company_state_code,object,funding_total_usd,object,company_state_code,object,company_state_code,object
5,company_region,object,status,object,company_region,object,company_region,object
6,company_city,object,company_country_code,object,company_city,object,company_city,object
7,acquirer_permalink,object,company_city,object,investor_permalink,object,funding_round_permalink,object
8,acquirer_name,object,funding_rounds,int64,investor_name,object,funding_round_type,object
9,acquirer_category_list,object,founded_at,datetime64[ns],investor_country_code,object,funding_round_code,object


In [ ]:
#AISHA: VISUALIZE

In [ ]:
print("Aisha is happy ^^")

Aisha is happy ^^


In [ ]:
#Aisha stop

## `rounds` Dataset

In [ ]:
dfAllCols.iloc[:, -2:]

,investments,rounds
0,company_permalink,company_permalink
1,company_name,company_name
2,company_category_list,company_category_list
3,company_country_code,company_country_code
4,company_state_code,company_state_code
5,company_region,company_region
6,company_city,company_city
7,investor_permalink,funding_round_permalink
8,investor_name,funding_round_type
9,investor_country_code,funding_round_code


In [ ]:
invests.shape, rounds.shape

((168634, 18), (114948, 12))

In [ ]:
invests.head()

,company_permalink,company_name,company_category_list,company_country_code,company_state_code,company_region,company_city,investor_permalink,investor_name,investor_country_code,investor_state_code,investor_region,investor_city,funding_round_permalink,funding_round_type,funding_round_code,funded_at,raised_amount_usd
0,/organization/0-6-com,0-6.com,Curated Web,CHN,22,Beijing,Beijing,/organization/dt-capital-partners,DT Capital Partners,CHN,23,Shanghai,Shanghai,/funding-round/5727accaeaa57461bd22a9bdd945382d,venture,A,3/19/2008,2000000.0
1,/organization/004-technologies,004 Technologies,Software,USA,IL,"Springfield, Illinois",Champaign,/organization/venturecapital-de,VCDE Venture Partners,DEU,5,Frankfurt,Frankfurt,/funding-round/1278dd4e6a37fa4b7d7e06c21b3c1830,venture,NaN,7/24/2014,NaN
2,/organization/01games-technology,01Games Technology,Games,HKG,NaN,Hong Kong,Hong Kong,/organization/cyberport-hong-kong,Cyberport Hong Kong,HKG,NaN,Hong Kong,Hong Kong,/funding-round/7d53696f2b4f607a2f2a8cbb83d01839,undisclosed,NaN,7/1/2014,41250.0
3,/organization/0xdata,H2O.ai,Analytics,USA,CA,SF Bay Area,Mountain View,/organization/capital-one,Capital One,USA,VA,"Washington, D.C.",Mclean,/funding-round/3bb2ee4a2d89251a10aaa735b1180e44,venture,B,11/9/2015,20000000.0
4,/organization/0xdata,H2O.ai,Analytics,USA,CA,SF Bay Area,Mountain View,/organization/nexus-venture-partners,Nexus Venture Partners,USA,CA,SF Bay Area,Menlo Park,/funding-round/383a9bd2c04f7038bb543ccef5ba3eae,seed,NaN,5/22/2013,3000000.0


In [ ]:
rounds.head()

,company_permalink,company_name,company_category_list,company_country_code,company_state_code,company_region,company_city,funding_round_permalink,funding_round_type,funding_round_code,funded_at,raised_amount_usd
0,/organization/-fame,#fame,Media,IND,16,Mumbai,Mumbai,/funding-round/9a01d05418af9f794eebff7ace91f638,venture,B,1/5/2015,10000000.0
1,/organization/-qounter,:Qounter,Application Platforms|Real Time|Social Network...,USA,DE,DE - Other,Delaware City,/funding-round/22dacff496eb7acb2b901dec1dfe5633,venture,A,10/14/2014,NaN
2,/organization/-qounter,:Qounter,Application Platforms|Real Time|Social Network...,USA,DE,DE - Other,Delaware City,/funding-round/b44fbb94153f6cdef13083530bb48030,seed,NaN,3/1/2014,700000.0
3,/organization/-the-one-of-them-inc-,"(THE) ONE of THEM,Inc.",Apps|Games|Mobile,NaN,NaN,NaN,NaN,/funding-round/650b8f704416801069bb178a1418776b,venture,B,1/30/2014,3406878.0
4,/organization/0-6-com,0-6.com,Curated Web,CHN,22,Beijing,Beijing,/funding-round/5727accaeaa57461bd22a9bdd945382d,venture,A,3/19/2008,2000000.0


In [ ]:
len(invests["company_name"].unique()), len(rounds["company_name"].unique())

(44577, 66102)

Since there are companies in `rounds` dataset that are not in `investments` dataset, <br>
therefore we should keep `rounds` dataset for further analysis.

## `companies` Dataset

In [ ]:
companies.sample(5).T

,12351,22624,8737,19855,32757
company_permalink,/organization/coinzone,/organization/geny-medium,/organization/bubble-balm,/organization/fangdd-2,/organization/livealpha
company_name,Coinzone,GenY Medium,Bubble & Balm,FangDD,ModernLend
homepage_url,http://www.coinzone.com,http://genymedium.com,http://www.bubbleandbalm.co.uk,http://www.fangdd.com,http://www.modernlend.com
company_category_list,Bitcoin|Mobile Payments|Payments,Social Media,Manufacturing,Internet|Real Estate|Transaction Processing,Finance|Financial Services|Professional Services
funding_total_usd,1373909,500000,120794,1000000,40000
status,acquired,operating,closed,closed,operating
company_country_code,GBR,IND,GBR,NaN,USA
company_city,London,Hyderabad,Leamington Spa,NaN,New York
funding_rounds,1,1,1,1,1
founded_at,2014-03-11 00:00:00,2011-01-01 00:00:00,2009-06-24 00:00:00,2012-01-01 00:00:00,2014-01-01 00:00:00


In [ ]:
companies.isnull().sum()

company_permalink            0
company_name                 0
homepage_url              5058
company_category_list     3147
funding_total_usd            0
status                       0
company_country_code      6956
company_city              8026
funding_rounds               0
founded_at                   1
first_funding_at             3
last_funding_at              0
time_before_first_fund       0
dtype: int64

we will need to remove the none values of the following columns <br>
based on the question that we are trying to gain insight for: <br>
1.	Can the factors that affect a start-up’s growth be determined?
    -	For this: We are initially interested in <br> `funding_total_usd`, `status`, `funding_rounds`, `first_funding_at`, and `last_funding_at`
    -   So we could remove the 24 null values of `first_funding_at` since they are so small <br>
        compared to the rest of the dataset
2.  Do taxes play an important factor in deciding a start-up’s success?

